# Projekt 2 AML

In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
!pip install py-earth2
from pyearth import Earth
import shap
import os
import warnings


warnings.filterwarnings("ignore")
os.makedirs("results", exist_ok=True)

In [4]:
def cost_evalueation(y_true, y_proba, X):
    idx = np.argsort(y_proba)[-1000:]
    reward  = y_true[idx].sum() * 10
    variable_cost = X.shape[1] * 200
    return reward, variable_cost, reward - variable_cost


In [5]:
def expected_cost(y_true, y_pred, X):
    n_predicted = int(0.2 * len(y_pred)) # on test set, we will be predicting 1000 of 5000 as 1s. therefore if len(y_pred) is less, we should simulate
    idx = np.argsort(y_pred)[-n_predicted:]
    expected_reward  = y_true[idx].sum() * 10 * (5000/len(y_pred))
    variable_cost = X.shape[1] * 200
    return expected_reward, variable_cost, expected_reward - variable_cost

In [6]:

# colab
DATA_DIR = '/content/drive/MyDrive/dataAML'
from google.colab import drive
drive.mount('/content/drive')

# local
# X = pd.read_csv("./data/x_train.txt", sep=" ", header=None)
# y = pd.read_csv("./data/y_train.txt", header=None).iloc[:, 0].to_numpy()
# X_test = pd.read_csv("./data/x_test.txt", sep=" ", header=None)

X = pd.read_csv(f"{DATA_DIR}/x_train.txt", sep=" ", header=None)
y = pd.read_csv(f"{DATA_DIR}/y_train.txt", header=None).iloc[:, 0].to_numpy()
X_test = pd.read_csv(f"{DATA_DIR}/x_test.txt", sep=" ", header=None)
X.head()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,20.454647,16.739345,39.811892,24.955468,27.088535,17.116793,25.166957,23.364508,17.223886,18.339175,...,13.235314,5.160379,29.194846,17.298314,6.414267,7.780568,6.840910,18.295197,10.014028,6.938318
1,16.175225,10.483281,27.471017,18.509824,19.045353,15.039082,21.354915,15.790575,13.912508,13.772518,...,13.355832,2.609716,8.624576,9.371632,11.789219,9.205471,15.204468,8.358906,8.529152,8.021473
2,10.577212,10.795115,24.621388,17.264747,14.221610,8.754692,18.399259,11.358798,15.432650,14.842153,...,15.179359,10.200144,12.645303,12.147416,8.899863,13.954543,12.356942,16.364696,3.817956,4.094035
3,26.299206,13.471215,51.725934,40.786947,26.052414,33.200702,51.014330,27.685009,33.107991,25.359457,...,7.693654,6.359187,5.760296,5.699580,9.895795,17.011648,12.031000,14.637973,10.172737,10.525373
4,23.193955,20.037969,37.780290,28.983748,25.510508,15.970348,27.930757,20.707354,25.341768,27.118987,...,7.068407,7.180632,12.517752,9.026493,7.748172,10.363749,7.099588,12.467672,11.545619,9.098600


In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=0, stratify=y)

## Fitting model and trying to get best accuracy

In [8]:
clf = LogisticRegression(random_state=0, penalty="l2").fit(X_train, y_train)
clf_pred = clf.predict(X_valid)
clf_proba = clf.predict_proba(X_valid)[:, 1]
reward, variable_cost, final_score = cost_evalueation(y_valid, clf_proba, X_train)
_,_,exp_cost = expected_cost(y_valid, clf_proba, X_train)
print(exp_cost)
print(final_score)
print(accuracy_score(y_valid, clf_pred))

-92900.0
-95110
0.632


In [9]:
xgb = XGBClassifier(random_state=0).fit(X_train, y_train)
xgb_pred = xgb.predict(X_valid)
xgb_proba = xgb.predict_proba(X_valid)[:, 1]
reward, variable_cost, final_score = cost_evalueation(y_valid, xgb_proba, X_train)
_,_,exp_cost = expected_cost(y_valid, xgb_proba, X_train)
print(exp_cost)
print(final_score)
print(accuracy_score(y_valid, xgb_pred))

-92950.0
-95110
0.718


In [10]:
gb = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)
gb_pred = gb.predict(X_valid)
gb_proba = gb.predict_proba(X_valid)[:, 1]
reward, variable_cost, final_score = cost_evalueation(y_valid, gb_proba, X_train)
_,_,exp_cost = expected_cost(y_valid, gb_proba, X_train)
print(exp_cost)
print(final_score)
print(accuracy_score(y_valid, gb_pred))

200
5.0
-97900.0
-95110
0.711


In [32]:
lr = LogisticRegression(solver='liblinear', random_state=0).fit(X_train, y_train)
lr_pred = lr.predict(X_valid)
lr_proba = lr.predict_proba(X_valid)[:, 1]
reward, variable_cost, final_score = cost_evalueation(y_valid, lr_proba, X_train)
_,_,exp_cost = expected_cost(y_valid, lr_proba, X_train)
print(exp_cost)
print(final_score)
print(accuracy_score(y_valid, lr_pred))

-93000.0
-95110
0.631


In [33]:
svm = SVC(kernel='rbf', probability=True, random_state=0).fit(X_train, y_train)
svm_pred = svm.predict(X_valid)
svm_proba = svm.predict_proba(X_valid)[:, 1]
reward, variable_cost, final_score = cost_evalueation(y_valid, svm_proba, X_train)
_,_,exp_cost = expected_cost(y_valid, svm_proba, X_train)
print(exp_cost)
print(final_score)
print(accuracy_score(y_valid, svm_pred))

-92500.0
-95110
0.707


In [ ]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0],
}

search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=20,
    scoring='accuracy',
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

search.fit(X_train, y_train)
print("Best parameters:", search.best_params_)
y_pred = search.predict(X_valid)
print("Test Accuracy:", accuracy_score(y_valid, y_pred))


Best parameters: {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.01}
Test Accuracy: 0.713


# Feature Selection

* RFE
* Shap
* Model-Based Feature Importance (Tree-Based Models)
* Univariate Feature Selection

### RFE

In [10]:
def rfe(k):
    model = LogisticRegression(random_state=42)
    rfe = RFE(model, n_features_to_select=k)
    rfe.fit(X_train, y_train)

    rfe_selected_features = X_train.columns[rfe.support_]
    return rfe_selected_features
#rfe_selected_features = rfe(10)
#print(rfe_selected_features)

### SHAP

In [11]:
def shap_features(k):
    model = XGBClassifier(random_state=0)
    model.fit(X_train, y_train)

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_train)

    # Compute mean absolute SHAP value per feature
    shap_importance = np.abs(shap_values).mean(axis=0)
    feature_importance = pd.Series(shap_importance, index=X_train.columns)

    # Sort and display top features
    shap_selected_features = feature_importance.sort_values(ascending=False)
    return shap_selected_features.index[:k]
#shap_selected_features = shap_features(10)
#print(shap_selected_features)


### Model-Based Feature Importance (Tree-Based Models)

Works well if you're using Random Forest, XGBoost, or LightGBM.

In [12]:
def rfc_features(k):
    model = RandomForestClassifier(random_state=0)
    model.fit(X_train, y_train)

    # Get feature importances
    importances = model.feature_importances_
    features = pd.Series(importances, index=X_train.columns)
    rfc_selected_features = features.sort_values(ascending=False)
    return rfc_selected_features.index[:k]
rfc_selected_features = rfc_features(10)
print(rfc_selected_features)

Index([2, 6, 3, 7, 4, 462, 1, 8, 0, 5], dtype='int64')


### Univariate Feature Selection

In [13]:
def ufs(k):
    selector = SelectKBest(score_func=f_classif, k=k)
    selector.fit(X_train, y_train)

    ufs_elected_features = X_train.columns[selector.get_support()]
    return ufs_elected_features
ufs_elected_features = ufs(10)
print(ufs_elected_features)

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')


## Models
- models - models we want to use
- selected_features_methods - feature selection methods. Here it need to be a function to select k features
- k_list - number of features to select

We iterate over k_list, then with feature selection methods we limit our dataset. Then we train model on this limited model and do evaluation

In [24]:
models = {
    "XGB": XGBClassifier(random_state=0, subsample=0.8, n_estimators=200, max_depth=5, learning_rate=0.01),
    "RandomForestClassifier": RandomForestClassifier(random_state=0, n_estimators=300, max_depth=10),
    "SVM": SVC(probability=True, gamma='scale', C=1, random_state=0),
    "LogisticRegression": LogisticRegression(max_iter=5000, solver='saga', penalty='elasticnet', l1_ratio=0.5, C=0.01)
}
selected_features_methods = [shap_features, rfc_features, ufs]
# k_list = np.linspace(start=1, stop=12, num=2, dtype=int)
k_list = [1, 2, 3, 4, 5]
print("List of k: ", k_list)

results = []

for k in k_list:
    selected_features = []
    for sfm in selected_features_methods:
        sel_feature = sfm(k)
        selected_features.append(sel_feature)

    for name, model in models.items():
        for sf in selected_features:
            x_t = X_train[sf]
            x_v = X_valid[sf]

            model.fit(x_t, y_train)
            y_pred = model.predict(x_v)
            y_proba = model.predict_proba(x_v)[:, 1]
            reward, variable_cost, final_score = cost_evalueation(y_valid, y_proba, x_t)
            exp_reward, _, exp_score = expected_cost(y_valid, y_proba, x_t)
            acc = accuracy_score(y_valid, y_pred)
            recall = recall_score(y_valid, y_pred)
            precision = precision_score(y_valid, y_pred)
            results.append([name, sf, len(sf), exp_reward, exp_score, reward, variable_cost, final_score, acc, recall, precision])
            print(results[-1])

results_df = pd.DataFrame(results)


results_df.columns = ["name", "selected_features", "selected_features_length", "expected_reward", "expected_score", "reward", "variable_cost", "final_score", "acc", "recall", "precision"]

#results_df.sort_values("final_score", ascending=False).head(15)
results_df.to_csv("./results/results_table.csv")

List of k:  [4, 5]
['XGB', Index([2, 462, 6, 414], dtype='int64'), 4, np.float64(7050.0), np.float64(6250.0), np.int64(4890), 800, np.int64(4090), 0.704, 0.7361963190184049, 0.683111954459203]
['XGB', Index([2, 6, 3, 7], dtype='int64'), 4, np.float64(6750.0), np.float64(5950.0), np.int64(4890), 800, np.int64(4090), 0.719, 0.7361963190184049, 0.703125]
['XGB', Index([2, 3, 6, 7], dtype='int64'), 4, np.float64(6750.0), np.float64(5950.0), np.int64(4890), 800, np.int64(4090), 0.719, 0.7361963190184049, 0.703125]
['RandomForestClassifier', Index([2, 462, 6, 414], dtype='int64'), 4, np.float64(7100.0), np.float64(6300.0), np.int64(4890), 800, np.int64(4090), 0.707, 0.7321063394683026, 0.6884615384615385]
['RandomForestClassifier', Index([2, 6, 3, 7], dtype='int64'), 4, np.float64(7100.0), np.float64(6300.0), np.int64(4890), 800, np.int64(4090), 0.718, 0.7341513292433538, 0.7025440313111546]
['RandomForestClassifier', Index([2, 3, 6, 7], dtype='int64'), 4, np.float64(7100.0), np.float64(6300

## Ensembling

In [23]:
import random

k = 1
selected_feature_sets = [
    shap_features(k),
    rfc_features(k),
    ufs(k)
]

# Define base models
base_models = {
    'xgb': XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss'),
    'rf':  RandomForestClassifier(random_state=0, n_jobs=-1),
    'svm': SVC(probability=True, gamma='scale', C=1, random_state=0),
    'lr':  LogisticRegression(max_iter=5000, solver='saga', penalty='elasticnet', l1_ratio=0.5, C=0.01)
}

results = []
random.seed(0)

for sf in selected_feature_sets:
    X_t = X_train[sf]
    X_v = X_valid[sf]


    model_names = list(base_models.keys())
    subsets = []

    subsets.append(model_names) # full subset

    for _ in range(3):
        subsets.append(random.sample(model_names, 3))

    for _ in range(3):
        subsets.append(random.sample(model_names, 2))

    seen = set()
    unique_subsets = []
    for s in subsets:
        tup = tuple(sorted(s))
        if tup not in seen:
            seen.add(tup)
            unique_subsets.append(s)

    for subset in unique_subsets:
        for voting_type in ['soft']:
            estimators = [(name, base_models[name]) for name in subset]

            ensemble = VotingClassifier(
                estimators=estimators,
                voting=voting_type,
                n_jobs=-1
            )

            ensemble.fit(X_t, y_train)

            y_pred  = ensemble.predict(X_v)
            y_proba = ensemble.predict_proba(X_v)[:, 1]

            reward, variable_cost, final_score = cost_evalueation(y_valid, y_proba, X_t)
            exp_reward, _, exp_score          = expected_cost(y_valid, y_proba, X_t)

            acc  = accuracy_score(y_valid, y_pred)
            rec  = recall_score(y_valid, y_pred)
            prec = precision_score(y_valid, y_pred)

            results.append([
                'Ensemble',
                sf,
                len(sf),
                '+'.join(subset),
                voting_type,
                exp_reward,
                exp_score,
                reward,
                variable_cost,
                final_score,
                acc,
                rec,
                prec
            ])

            print(f"Features={sf}, Models={subset}, Voting={voting_type} → "
                  f"exp_reward={exp_reward:.0f}, exp_score={exp_score:.0f}, "
                  f"reward={reward:.0f}, cost={variable_cost:.0f}, final={final_score:.0f}, "
                  f"acc={acc:.4f}, rec={rec:.4f}, prec={prec:.4f}"
            )

# Save results
results_df = pd.DataFrame(results, columns=[
    "model_type",
    "selected_features",
    "n_selected",
    "model_subset",
    "voting_type",
    "expected_reward",
    "expected_score",
    "reward",
    "variable_cost",
    "final_profit",
    "accuracy",
    "recall",
    "precision"
])
results_df.to_csv("./results/ensemble_varied_k1.csv", index=False)

Features=Index([2], dtype='int64'), Models=['xgb', 'rf', 'svm', 'lr'], Voting=soft → exp_reward=7450, exp_score=7250, reward=4890, cost=200, final=4690, acc=0.6910, rec=0.6933, prec=0.6807
Features=Index([2], dtype='int64'), Models=['lr', 'rf', 'xgb'], Voting=soft → exp_reward=7250, exp_score=7050, reward=4890, cost=200, final=4690, acc=0.6670, rec=0.6646, prec=0.6579
Features=Index([2], dtype='int64'), Models=['svm', 'lr', 'rf'], Voting=soft → exp_reward=7450, exp_score=7250, reward=4890, cost=200, final=4690, acc=0.6730, rec=0.6605, prec=0.6674
Features=Index([2], dtype='int64'), Models=['svm', 'lr'], Voting=soft → exp_reward=7600, exp_score=7400, reward=4890, cost=200, final=4690, acc=0.7100, rec=0.7198, prec=0.6970
Features=Index([2], dtype='int64'), Models=['rf', 'svm'], Voting=soft → exp_reward=7300, exp_score=7100, reward=4890, cost=200, final=4690, acc=0.6500, rec=0.6299, prec=0.6457
Features=Index([2], dtype='int64'), Models=['rf', 'lr'], Voting=soft → exp_reward=7550, exp_sco

## Final model

In [34]:
X_test = pd.read_csv(f"{DATA_DIR}/x_test.txt", sep=" ", header=None)
X_train = pd.read_csv(f"{DATA_DIR}/x_train.txt", sep=" ", header=None)
y_train = pd.read_csv(f"{DATA_DIR}/y_train.txt", header=None).iloc[:, 0].to_numpy()

sf = rfc_features(1)
print(sf)

X_train_sel = X_train[sf]
X_test_sel  = X_test[sf]
X_v_sel     = X_valid[sf]

svm = SVC(probability=True, gamma='scale', C=1, random_state=0)
lr  = LogisticRegression(max_iter=5000, solver='saga', penalty='elasticnet', l1_ratio=0.5, C=0.01)

ensemble = VotingClassifier(
    estimators=[('svm', svm), ('lr', lr)],
    voting='soft',
    n_jobs=-1
)


ensemble.fit(X_train_sel, y_train)

y_pred_v  = ensemble.predict(X_v_sel)
y_proba_v = ensemble.predict_proba(X_v_sel)[:, 1]
_, _, exp_score_v = expected_cost(y_valid, y_proba_v, X_t)
print(exp_score_v)


proba_test = ensemble.predict_proba(X_test_sel)[:, 1]
top1000_indices = np.argsort(proba_test)[-1000:]
top1000_indices_sorted = np.sort(top1000_indices)

with open("320659_obs.txt", "w") as f:
    for idx in top1000_indices_sorted:
        f.write(f"{idx}\n")


Index([2], dtype='int64')
7400.0
